In [ ]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class codats(nn.Module):
    def __init__(self, common_dim, hidden_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(codats, self).__init__()

        # hyperparameters
        self.input_dim = common_dim
        self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.layers = nn.Sequential(
                nn.Conv1d(1, hidden_dim, kernel_size=8, padding=4, bias=False),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),

                nn.Conv1d(hidden_dim, 2*hidden_dim, kernel_size=5, padding=2, bias=False),
                nn.BatchNorm1d(2*hidden_dim),
                nn.ReLU(),

                nn.Conv1d(2*hidden_dim, hidden_dim, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),

                nn.AdaptiveAvgPool1d(1),
            )
        

        self.demo_proj_main = nn.Linear(12, self.hidden_dim)
        self.demo_proj = nn.Linear(12, self.hidden_dim)
        self.Linear = nn.Linear(self.hidden_dim, 1)
        self.output = nn.Linear(self.input_dim, self.output_dim)

        # adversal方法中的域分类器  
        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(self.hidden_dim, self.hidden_dim))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(self.hidden_dim))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(hidden_dim, 2))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.FC_embed = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()
        self.to_MMD = nn.Linear(self.hidden_dim, 1)

    def forward(self, input, lens, alpha, is_teacher):
        lens = lens.to('cpu')
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)# input Tensor : 256 * 48 * 76
        assert(self.d_model % self.MHD_num_head == 0)
        
        codats_embeded_input = self.layers(input[:,:,0].squeeze().unsqueeze(1)).unsqueeze(1)
        # print(f'input[:,:,i].shape is {input[:,:,0].shape}')
        for i in range(1, feature_dim):
            embeded_input = self.layers(input[:,:,i].squeeze().unsqueeze(1)).unsqueeze(1)
            codats_embeded_input = torch.cat((codats_embeded_input, embeded_input), 1)
            # print(f'docats_embeded_input.shape is {codats_embeded_input.shape}')
        codats_embeded_input = codats_embeded_input.squeeze()
        # print(f'codats_embeded_input.shape is {codats_embeded_input.shape}')
        # GRU_embeded_input = self.GRUs[0](pack_padded_sequence(input[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
        # for i in range(feature_dim-1):
        #     embeded_input = self.GRUs[i+1](pack_padded_sequence(input[:,:,i+1].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
        #     GRU_embeded_input = torch.cat((GRU_embeded_input, embeded_input), 1)
        # print(f"GRU_embeded_input.shape is {GRU_embeded_input.shape}")
        


        if is_teacher: # 来自源数据集
            common_input = codats_embeded_input[:, 0, :]
            for i in range(1, feature_dim):
                common_input = common_input + codats_embeded_input[:, i, :]  
            print(f"common_input1.shape is {common_input.shape}")
            common_input = torch.squeeze(common_input, 1) # batch * hidden
            reverse_input = ReverseLayerF.apply(common_input, alpha)
            print(f"common_input2.shape is {common_input.shape}")
            domain_output = self.domain_classifier(reverse_input)

            posi_input = self.dropout(codats_embeded_input) # batch_size * d_input + d_input_diff * hidden_dim
            
            contexts = self.Linear(posi_input).squeeze()# b i
            output = self.output(self.dropout(contexts))# b 1
            output = self.sigmoid(output)
            return output, domain_output, contexts
        else: # 来自目标数据集，主要是为了混淆domain classifier
            common_input = codats_embeded_input[:, 0, :]
            for i in range(1, feature_dim):
                common_input = common_input + codats_embeded_input[:, i, :]  
            common_input = torch.squeeze(common_input, 1) # batch * hidden
            reverse_input = ReverseLayerF.apply(common_input, alpha)
            domain_output = self.domain_classifier(reverse_input)
            return domain_outputclass ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class codats(nn.Module):
    def __init__(self, common_dim, hidden_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(codats, self).__init__()

        # hyperparameters
        self.input_dim = common_dim
        self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.layers = nn.Sequential(
                nn.Conv1d(1, hidden_dim, kernel_size=8, padding=4, bias=False),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),

                nn.Conv1d(hidden_dim, 2*hidden_dim, kernel_size=5, padding=2, bias=False),
                nn.BatchNorm1d(2*hidden_dim),
                nn.ReLU(),

                nn.Conv1d(2*hidden_dim, hidden_dim, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),

                nn.AdaptiveAvgPool1d(1),
            )
        

        self.demo_proj_main = nn.Linear(12, self.hidden_dim)
        self.demo_proj = nn.Linear(12, self.hidden_dim)
        self.Linear = nn.Linear(self.hidden_dim, 1)
        self.output = nn.Linear(self.input_dim, self.output_dim)

        # adversal方法中的域分类器  
        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(self.hidden_dim, self.hidden_dim))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(self.hidden_dim))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(hidden_dim, 2))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.FC_embed = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()
        self.to_MMD = nn.Linear(self.hidden_dim, 1)

    def forward(self, input, lens, alpha, is_teacher):
        lens = lens.to('cpu')
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)# input Tensor : 256 * 48 * 76
        assert(self.d_model % self.MHD_num_head == 0)
        
        codats_embeded_input = self.layers(input[:,:,0].squeeze().unsqueeze(1)).unsqueeze(1)
        # print(f'input[:,:,i].shape is {input[:,:,0].shape}')
        for i in range(1, feature_dim):
            embeded_input = self.layers(input[:,:,i].squeeze().unsqueeze(1)).unsqueeze(1)
            codats_embeded_input = torch.cat((codats_embeded_input, embeded_input), 1)
            # print(f'docats_embeded_input.shape is {codats_embeded_input.shape}')
        codats_embeded_input = codats_embeded_input.squeeze()
        # print(f'codats_embeded_input.shape is {codats_embeded_input.shape}')
        # GRU_embeded_input = self.GRUs[0](pack_padded_sequence(input[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
        # for i in range(feature_dim-1):
        #     embeded_input = self.GRUs[i+1](pack_padded_sequence(input[:,:,i+1].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
        #     GRU_embeded_input = torch.cat((GRU_embeded_input, embeded_input), 1)
        # print(f"GRU_embeded_input.shape is {GRU_embeded_input.shape}")
        


        if is_teacher: # 来自源数据集
            common_input = codats_embeded_input[:, 0, :]
            for i in range(1, feature_dim):
                common_input = common_input + codats_embeded_input[:, i, :]  
            print(f"common_input1.shape is {common_input.shape}")
            common_input = torch.squeeze(common_input, 1) # batch * hidden
            reverse_input = ReverseLayerF.apply(common_input, alpha)
            print(f"common_input2.shape is {common_input.shape}")
            domain_output = self.domain_classifier(reverse_input)

            posi_input = self.dropout(codats_embeded_input) # batch_size * d_input + d_input_diff * hidden_dim
            
            contexts = self.Linear(posi_input).squeeze()# b i
            output = self.output(self.dropout(contexts))# b 1
            output = self.sigmoid(output)
            return output, domain_output, contexts
        else: # 来自目标数据集，主要是为了混淆domain classifier
            common_input = codats_embeded_input[:, 0, :]
            for i in range(1, feature_dim):
                common_input = common_input + codats_embeded_input[:, i, :]  
            common_input = torch.squeeze(common_input, 1) # batch * hidden
            reverse_input = ReverseLayerF.apply(common_input, alpha)
            domain_output = self.domain_classifier(reverse_input)
            return domain_output